# Data --> CSP-Structure

## How should the Data-Structure look like?

It's a Pandas Dataframe with all relevant Information.
| id | size | variables | domains | constraints |
|----|------|-----------|---------|-------------|
| lgp-test-5x6-16 | 5*6 | ['name', 'nationalities', 'book genres', 'lunch', 'color', 'animals'] | {'name': [`Peter`, `Alice`, `Bob`, `Eric`, `Arnold`], 'nationalities': [`norwegian`, `german`, `dane`, `brit`, `swede`], 'book genres': [`fantasy`, `biography`, `romance`, `mystery`, `science fiction`], 'lunch': [`stir fry`, `grilled cheese`, `pizza`, `spaghetti`, `stew`], 'color': [`red`, `green`, `blue`, `yellow`, `white`], 'animals': [`bird`, `dog`, `cat`, `horse`, `fish`]} | [IsContraint('fantasy', 'norwegian'), NextToConstraint('cat', 'biography'), IsConstraint('German', 'Bob'), IsConstraint('yellow', 'Bob'), IsConstraint('Green', 'Peter'), LeftOrRightConstraint('Dane', 'Pizza', distance=1), LeftConstraint('blue', 'Dane'), LeftConstraint('grilled Cheese', 'Norwegian'), ..., LeftConstraint('fish', 'science fiction', distance=0), ...,  RightConstraint(x,y), RightConstraint(x,y, distance=0)] |

In [8]:
import pandas as pd
import numpy as np
import re

columns = ["id", "size", "variables", "Domains", "Constraints"]
parsed = pd.DataFrame(columns=columns)

# df = pd.read_parquet("data/Gridmode-00000-of-00001.parquet")
df = pd.read_parquet("hf://datasets/allenai/ZebraLogicBench/grid_mode/test-00000-of-00001.parquet")


In [14]:
for index, text in enumerate(df["solution"]):
  # The Key "header" contains all our Variables
  parsed.loc[index, "variables"] = text.get("header", None)

  parsed.loc[index, "id"] = df.loc[index, "id"]
  parsed.loc[index, "size"] = df.loc[index, "size"]

print(parsed)

                  id size                                          variables  \
0    lgp-test-5x6-16  5*6  [House, Name, Nationality, BookGenre, Food, Co...   
1    lgp-test-4x4-27  4*4   [House, Name, Occupation, BookGenre, PhoneModel]   
2    lgp-test-6x4-15  6*4        [House, Name, Children, MusicGenre, Height]   
3     lgp-test-6x5-2  6*5  [House, Name, Mother, Children, Vacation, Book...   
4    lgp-test-2x2-33  2*2                                 [House, Name, Pet]   
..               ...  ...                                                ...   
995   lgp-test-3x4-1  3*4               [House, Name, Animal, Cigar, Mother]   
996  lgp-test-5x5-39  5*5  [House, Name, Birthday, Mother, Occupation, Ha...   
997  lgp-test-2x2-25  2*2                            [House, Name, Vacation]   
998  lgp-test-2x4-34  2*4       [House, Name, PhoneModel, Occupation, Color]   
999  lgp-test-5x3-14  5*3                        [House, Name, Color, Cigar]   

                                       

In [12]:
text = df["puzzle"]

for index, problem in enumerate(text):
  vars = parsed.loc[index, "variables"]

  # output Directionairy
  results = {}

  i = 0
  # "House" doesn't have given Values, so we generate them ourselves
  if(vars[i] == "House"):
    houses = []

    # Generates the Domain for houses
    for h_Count in range(int(df["size"][index][0])):
      houses.append(h_Count+1)
      results["House"] = houses
    i = i+1

  for line in problem.splitlines():
    if line.strip().startswith("-"):

      # Filters all Expressions, that are in Between of '
      # resulting List gets addedd as a new Domain
      names = re.findall(r"`([^`]*)`", line)
      results[vars[i]] = names
      i = i+1

  parsed.at[index, "Domains"] = results
print(parsed["Domains"])


0      {'House': [1, 2, 3, 4, 5], 'Name': ['Peter', '...
1      {'House': [1, 2, 3, 4], 'Name': ['Alice', 'Eri...
2      {'House': [1, 2, 3, 4, 5, 6], 'Name': ['Bob', ...
3      {'House': [1, 2, 3, 4, 5, 6], 'Name': ['Bob', ...
4      {'House': [1, 2], 'Name': ['Eric', 'Arnold'], ...
                             ...                        
995    {'House': [1, 2, 3], 'Name': ['Arnold', 'Peter...
996    {'House': [1, 2, 3, 4, 5], 'Name': ['Alice', '...
997    {'House': [1, 2], 'Name': ['Arnold', 'Eric'], ...
998    {'House': [1, 2], 'Name': ['Eric', 'Arnold'], ...
999    {'House': [1, 2, 3, 4, 5], 'Name': ['Eric', 'A...
Name: Domains, Length: 1000, dtype: object
